<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab10/Instruction%20Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


# 지시문 학습(Instruction Tuning)
GPT는 텍스트를 가지고 통째로 학습했기 때문에, 챗봇처럼 사용자의 요청에 따른 대답을 생성해내지는 못합니다. 사용자의 지시에 결과를 생성하도록 하기 위해, 우리는 지시문 학습을 사용합니다.

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/instruction-tuning-sd/flan_schematic.png)

학습 방법은 간단한데, `지시문 -> 답변` 순서로 되어있는 텍스트 데이터를 가져와서 GPT를 파인튜닝하면 됩니다. 하지만 몇가지 유의할 점이 있습니다.

1. 지시문과 답변을 명확히 구분해주어야 합니다. 또한 답변이 끝나는 부분에 스페셜 토큰을(eos)을 넣어서 답변이 끝난다는 것을 알려주어야 합니다.
2. 지시문 부분은 label에 -100을 넣어서, 답변을 생성하는 부분만 학습되도록 합니다.
3. packing 기법을 활용해도 좋습니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# kogpt에서는 </s> 토큰을 eos token으로 쓸 예정입니다.
tokenizer.eos_token_id = 1

(…)/kogpt2-base-v2/resolve/main/config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

(…)gpt2-base-v2/resolve/main/tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [ ]:
# 이준범님이 지식인에서 크롤링한 koalpaca 데이터셋을 사용해보겠습니다.
# https://huggingface.co/datasets/beomi/KoAlpaca-v1.1a

from datasets import load_dataset

dataset = load_dataset("beomi/KoAlpaca-v1.1a", split="train")
dataset[0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [ ]:
import torch
from tqdm.auto import tqdm


instruction_format = """
### 질문: {instruction}

### 답변:"""

def pack_texts(items, max_seq_length):
  packed_tokens = []
  packed_labels = []

  eos_token_id = tokenizer.eos_token_id

  for item in tqdm(items):
    instruction = instruction_format.format(instruction=item["instruction"])
    output = item["output"]

    instruction = tokenizer.encode(instruction)
    output = tokenizer.encode(output)

    packed_tokens.extend(instruction + output)
    packed_tokens.append(eos_token_id)

    packed_labels.extend([-100] * len(instruction) + output)
    packed_labels.append(eos_token_id)

    while len(packed_tokens) > max_seq_length:
      yield {
          "input_ids": torch.tensor(
              packed_tokens[ : max_seq_length],
              dtype=torch.int64
              ),
          "labels": torch.tensor(
              packed_labels[1 : max_seq_length + 1],
              dtype=torch.int64
          )
      }
      packed_tokens = packed_tokens[max_seq_length:]
      packed_labels = packed_labels[max_seq_length:]


packed_tokens = pack_texts(
    dataset.select(range(4)),
    128
)

for i, pack in enumerate(packed_tokens):
  print("#", i)
  print(tokenizer.decode(pack["input_ids"], skip_special_tokens=False))
  print(pack["labels"])

  0%|          | 0/4 [00:00<?, ?it/s]

# 0

### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기
tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  9164, 13532, 15694, 11039,
        17752, 15380,  9679, 21154, 45941, 17415, 17752, 13961,  9679, 21154,
         9051,   375, 30895, 40921, 10066,  8022,  9552, 48397,  8711,  9033,
         9129, 20661, 11296,  9018, 45211,  9167, 35482, 25579,  9846, 32240,
         9164, 13532, 15694, 11039, 15380,  9679, 21154, 45941, 17415,  9306,
         9633, 10292, 12817,  7643,  9023, 47518, 10748,  9779, 13961,  9679,
        21154, 15957,  9164, 13532, 17752, 15380, 13748, 

출력 예시
```
# 0

### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다.

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요?

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기
tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  9164, 13532, 15694, 11039,
        17752, 15380,  9679, 21154, 45941, 17415, 17752, 13961,  9679, 21154,
         9051,   375, 30895, 40921, 10066,  8022,  9552, 48397,  8711,  9033,
         9129, 20661, 11296,  9018, 45211,  9167, 35482, 25579,  9846, 32240,
         9164, 13532, 15694, 11039, 15380,  9679, 21154, 45941, 17415,  9306,
         9633, 10292, 12817,  7643,  9023, 47518, 10748,  9779, 13961,  9679,
        21154, 15957,  9164, 13532, 17752, 15380, 13748, 11794, 45941, 17415,
        17752, 13961,  9679, 10542,   375, 16773,  9489, 12817,  7643,   401,
        29755,  9566,  9313, 19970,  7965,  7513,  8137,  9025,  9019,  7055,
         8084,   406,  9051,   375,  6889, 19307,  9566,  9313, 47160, 19970,
         7965,  7513,  8137,  9025, 32987, 10351, 15380, 11178])
```

In [ ]:
from pprint import pprint

batch_size = 1
max_seq_length = 1024

# 이번에는 양이 많지않아서 IterableDataset을 사용하지 않겠습니다.
packed_dataset = [pack for pack in pack_texts(dataset, max_seq_length)]

training_loader = torch.utils.data.DataLoader(
    packed_dataset,
    batch_size=batch_size,
    )

batch = next(iter(training_loader))

pprint(batch)

pprint(tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=False))

  0%|          | 0/21155 [00:00<?, ?it/s]

{'input_ids': tensor([[9051,  378,  378,  ...,  378,  378,  378]]),
 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100]])}
['\n'
 '### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n'
 '\n'
 '### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n'
 '\n'
 '식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. '
 '고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 '
 '부분입니다.\n'
 '\n'
 ' 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n'
 '\n'
 '고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. '
 '다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.</s> \n'
 '### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n'
 '\n'
 '### 답변: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 '
 '시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) '
 '만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 '
 '무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다.</s> \n'
 '### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  \n'
 '

In [ ]:
# 사전학습된 모델을 불러와서 파인튜닝해보겠습니다.
model = AutoModelForCausalLM.from_pretrained(model_id)
model

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
def count_parameters(model):
  params = sum(p.numel() for name, p in model.named_parameters())
  return params

print("total params", count_parameters(model))
print("model params", count_parameters(model.transformer.h))

total params 125164032
model params 85054464


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5) # 파인튜닝 때는 비교적 낮은 lr을 쓰는게 좋습니다

In [ ]:
from tqdm.auto import tqdm
import torch.nn.functional as F


def change_dict_device(x):
  return {k: v.to(device) for k, v in x.items()}

running_loss = 0.
last_loss = 0.
log_freq = 50
i = 0

# 3 에폭 학습하기
total_epochs = 3

model.train()
for epoch in range(total_epochs):
  for batch in tqdm(training_loader, desc=f"Epoch {epoch}"):
    i += 1

    optimizer.zero_grad()
    batch = change_dict_device(batch)

    logits = model(batch['input_ids']).logits

    logits = logits.view(-1, logits.shape[-1]) # [batch_size * sequence_length, vocab_size]
    labels = batch["labels"].view(-1) # [batch_size * sequence_length, vocab_size]

    loss = F.cross_entropy(logits, labels, ignore_index=-100)
    loss.backward()

    optimizer.step()

    running_loss += loss.item()
    if (i + 1) % log_freq == 0:
      last_loss = running_loss / log_freq
      print('batch {} loss: {}'.format(i + 1, last_loss))
      running_loss = 0.


Epoch 0:   0%|          | 0/3707 [00:00<?, ?it/s]

batch 50 loss: 3.891192717552185
batch 100 loss: 3.920224485397339
batch 150 loss: 3.934946646690369
batch 200 loss: 3.9597843360900877
batch 250 loss: 3.902907786369324
batch 300 loss: 3.9349954891204835
batch 350 loss: 3.920924334526062
batch 400 loss: 3.8784396553039553
batch 450 loss: 3.857242183685303
batch 500 loss: 3.965858497619629
batch 550 loss: 3.958516926765442
batch 600 loss: 3.909299793243408
batch 650 loss: 3.9337939977645875
batch 700 loss: 3.9497754049301146
batch 750 loss: 3.972288966178894
batch 800 loss: 3.8805693197250366
batch 850 loss: 3.927255835533142
batch 900 loss: 3.94356791973114
batch 950 loss: 3.928314414024353
batch 1000 loss: 3.911905312538147
batch 1050 loss: 3.8909434032440187
batch 1100 loss: 3.9624906873703
batch 1150 loss: 3.915590219497681
batch 1200 loss: 3.861804518699646
batch 1250 loss: 3.886471428871155
batch 1300 loss: 3.9502477979660036
batch 1350 loss: 3.918035235404968
batch 1400 loss: 3.8059070038795473
batch 1450 loss: 3.919195590019226

KeyboardInterrupt: ignored

In [ ]:
# 한번 테스트!
def get_answer(prefix):
  prefix = tokenizer([prefix], return_tensors="pt").to(device)
  outputs = model.generate(prefix['input_ids'], max_new_tokens=128, do_sample=True)

  print(tokenizer.decode(outputs[0]))

get_answer ("""
### 질문: 오늘의 날씨는 어때요?

### 답변: """
)

get_answer ("""
### 질문: 오늘의 날씨는 어때요?

### 답변: 아주 좋아요! 놀러가기 딱이죠

### 질문: 그러면 주말에 가족들과 수원에 놀러갈만한 곳이 있을까요?

### 답변:"""
)



### 질문: 오늘의 날씨는 어때요?

### 답변: 벚꽃은 음력 1월 1일에 뜨는다고 합니다. 그러나 이는 12월 31일 아침에 달이 뜨는 이유와는 별개의 문제입니다. 12월 1일 아침 달이 뜨는 데에는 달이 떠 있을 것입니다. 또한, 10월 1일 오후 2시에는 전국 어디에서나 볼 수 있는 벚꽃도 있으며, 각 지역의 벚꽃이 있는 날에 한해 4월 1일 밤 12시 30분에 잠자리에 들지 않는 것도 있습니다.</s>

### 질문: 오늘의 날씨는 어때요?

### 답변: 아주 좋아요! 놀러가기 딱이죠

### 질문: 그러면 주말에 가족들과 수원에 놀러갈만한 곳이 있을까요?

### 답변: 현재 가장 인기있는 곳은 광천 별장을 갖춘 광천 별장이 있습니다. 따라서, 가족들이나 수원에 머무르게 된다면 많은 곳에서 갈 수 있습니다. 다른 곳에 갈 때도 대부분 광천 근처에서는 꼭 가야 합니다. 이럴 경우 가장 많이 가는 것은 광천 별장에서 가장 넓은 평지에 있는 갈매기살호이호이호이호이(호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호이호호이호이호이호이


# Question

* 지시문 부분을 학습하지 않도록 한다고 했는데(-100으로 하는 등) 이대로 fine-tuning된 모델은 next token prediction에 대해서는 약해지는 경향이 생기게 되나요?